In [80]:
import pandas as pd 
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import unidecode
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
import re

In [105]:
# Load data from a text file
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            label, sentence = line.strip().split('\t', 1)  # Split on the first tab character
            data.append([sentence, int(label)])
    return pd.DataFrame(data, columns=['sentence', 'label'])

# file_path
file_path = 'TRAINING_DATA.txt'

#assign the data into pandas DF
data = load_data(file_path)

data.head()



,sentence,label
0,"Cuando conocí a Janice en 2013 , una familia n...",1
1,Hwang habló en Sur de este año por Southwest M...,0
2,Usted podría pensar Katy Perry y Robert Pattin...,1
3,Cualquiera que haya volado los cielos del crea...,1
4,"Bueno , este cantante tendrá un LARGO tiempo p...",1


In [3]:
data.shape

(17877, 2)

In [12]:
data['label'].value_counts()

label
0    8939
1    8938
Name: count, dtype: int64

In [4]:
#nltk resources 
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [106]:
#data cleaning, tokenization and stop words 
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove accents
    text = unidecode.unidecode(text)
    # Normalize numbers: Replace digits with a special token, e.g., "NUM"
    text = re.sub(r'\d+', 'NUM', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    #special characters 
    text = re.sub(r'[^\w\sáéíóúüñ¿¡]', '', text) 
    # Tokenize the text into words
    words = word_tokenize(text)
    # Define stop words for Spanish
    stop_words = set(stopwords.words('spanish'))
    # Remove stop words from the tokenized words
    words = [word for word in words if word not in stop_words]
    # Join the words back into a single string with spaces-
    return ' '.join(words)

data['cleaned_sentenced'] = data['sentence'].apply(preprocess_text)

In [62]:
data.head()

,sentence,label,cleaned_sentenced
0,"Cuando conocí a Janice en 2013 , una familia n...",1,conoci janice 2013 familia necesitaba 600 punt...
1,Hwang habló en Sur de este año por Southwest M...,0,hwang hablo sur ano southwest music and media ...
2,Usted podría pensar Katy Perry y Robert Pattin...,1,usted podria pensar katy perry robert pattinso...
3,Cualquiera que haya volado los cielos del crea...,1,cualquiera volado cielos creador escuchado act...
4,"Bueno , este cantante tendrá un LARGO tiempo p...",1,bueno cantante tendra largo tiempo sentir aun ...


In [107]:
# TF-IDF Vectorization function and logistic_regression
def tfidf_vectorize(data, text_column):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data[text_column])
    return X, vectorizer

# Logistic Regression Classification function
def logistic_regression_classification(X, y):
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,) #random_state=42)
    
    # Train the classifier
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return accuracy, report, classifier

# Main function to tie everything together
def main_tfidf_log_reg(data, text_column):
    # TF-IDF Vectorization
    X, vectorizer = tfidf_vectorize(data, text_column)
    y = data['label']

    # Classification
    accuracy, report, classifier = logistic_regression_classification(X, y)
    

    # Print class mapping
    class_mapping = classifier.classes_
    print(f'Class Mapping: {class_mapping}')
    
    # Output results
    print(f'Accuracy: {accuracy}')
    print(report)
    
    return vectorizer, classifier


vectorizer, classifier = main_tfidf_log_reg(data, 'cleaned_sentenced')

Class Mapping: [0 1]
Accuracy: 0.3615771812080537
              precision    recall  f1-score   support

           0       0.36      0.36      0.36      1774
           1       0.37      0.37      0.37      1802

    accuracy                           0.36      3576
   macro avg       0.36      0.36      0.36      3576
weighted avg       0.36      0.36      0.36      3576



### grandient boost algo with td-idf

In [77]:
from sklearn.ensemble import GradientBoostingClassifier

In [95]:
#Gradient boosting with td-idf
def gradient_boosting_classification(X, y):
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    return accuracy, report, classifier

# Main function to tie everything together
def main_gradient_boosting(data, text_column):
    # Count Vectorization
    X, vectorizer = tfidf_vectorize(data, text_column)
    y = data['label']
    
    # Classification
    accuracy, report, classifier = gradient_boosting_classification(X, y)
    
    # Output results
    print(f'Accuracy: {accuracy}')
    print(report)
    
    return vectorizer, classifier 

# Call the main function with gradient boosting
vectorizer, classifier = main_gradient_boosting(data, 'cleaned_sentenced')

Accuracy: 0.5050335570469798
              precision    recall  f1-score   support

           0       0.48      0.14      0.22      1751
           1       0.51      0.86      0.64      1825

    accuracy                           0.51      3576
   macro avg       0.50      0.50      0.43      3576
weighted avg       0.50      0.51      0.43      3576



In [ ]:
class_mapping = model.classes_
print(class_mapping)

### Cross- Validation

In [102]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords

# Ensure stop words are downloaded
import nltk
nltk.download('stopwords')

# Load your data into a DataFrame named `data`
# data = pd.read_csv('your_data.csv')  # Example loading

# Data definition
X = data['cleaned_sentenced']
y = data['label']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Spanish stop words
spanish_stop_words = stopwords.words('spanish')

# Define parameter grids for both TfidfVectorizer and GradientBoostingClassifier
param_grid = {
    'tfidf__max_features': [1000, 2000],  # Reduce number of combinations
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Reduce number of combinations
    'tfidf__max_df': [0.8, 0.9],  # Reduce number of combinations
    'tfidf__min_df': [0.01, 0.05],  # Reduce number of combinations
    'tfidf__stop_words': [None, spanish_stop_words],  # Use list of Spanish stop words
    'classifier__n_estimators': [50, 100],  # Reduce number of combinations
    'classifier__learning_rate': [0.1, 0.01]
}

# Create the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', GradientBoostingClassifier())
])

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1, error_score='raise')
try:
    grid_search.fit(X_train, y_train)
except Exception as e:
    print(f"An error occurred during grid search: {e}")

# Check if grid search was successful
if hasattr(grid_search, 'best_estimator_'):
    # Get best hyperparameters and best score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    # Print best hyperparameters and best score
    print("Best hyperparameters:", best_params)
    print("Best cross-validation score:", best_score)

    # Evaluate the best model on a holdout test set
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    test_score = accuracy_score(y_test, y_pred)
    print("Test score of the best model:", test_score)

    # Print the classification report
    print("Classification Report:\n", classification_report(y_test, y_pred))
else:
    print("Grid search did not complete successfully.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Fitting 5 folds for each of 128 candidates, totalling 640 fits
Best hyperparameters: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100, 'tfidf__max_df': 0.8, 'tfidf__max_features': 1000, 'tfidf__min_df': 0.01, 'tfidf__ngram_range': (1, 1), 'tfidf__stop_words': None}
Best cross-validation score: 0.5024121352258367
Test score of the best model: 0.5195749440715883
Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.11      0.19      1751
           1       0.52      0.91      0.66      1825

    accuracy                           0.52      3576
   macro avg       0.53      0.51      0.42      3576
weighted avg       0.53      0.52      0.43      3576

